# INTRO

This notebook is following 
https://www.kaggle.com/code/dimitriosroussis/electricity-price-forecasting-with-dnns-eda

Goal is to predict the _next hour_ electricity price. 
- EDA
- Feature engineering 

Modelling is done via several achitectures


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import tensorflow as tf
import xgboost as xgb
import os
import warnings
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, TimeDistributed, Flatten, Dropout, RepeatVector
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss, ccf
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from math import sqrt

%matplotlib inline

2023-09-27 17:16:05.704956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))


# Datasets
Here we have two .csv files 
- weather...csv _Hourly_ weather conditions
- energy...csv _Hourly_ energy generation, load, and price (energy generation is in MWh)

Weather data is for __5 Big cities__ that cover all the spain territory. 

In [4]:
df_weather = pd.read_csv("./weather_features.csv", parse_dates=["dt_iso"])
df_energy = pd.read_csv("./energy_dataset.csv", parse_dates=["time"])

In [6]:
df_energy.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [16]:
# check which mumber columns are all NaNs
for key, vals in df_energy.select_dtypes(include=np.number).items():
    if (len(vals[~np.isnan(np.array(vals))]) == 0):
        print(f"All nans for {key}")
# check which mumber columns are mostly 0
for key, vals in df_energy.select_dtypes(include=np.number).items():
    if (len(np.array(vals)[np.array(vals)==0.]) > 0.9 * len(np.array(vals))):
        print(f"All zeroes for {key}")
# drop these columns from the dataset
df_energy = df_energy.drop(
    [
        'generation fossil coal-derived gas','generation fossil oil shale', 
        'generation fossil peat', 'generation geothermal', 
        'generation hydro pumped storage aggregated', 'generation marine', 
        'generation wind offshore', 'forecast wind offshore eday ahead',
        'total load forecast', 'forecast solar day ahead',
        'forecast wind onshore day ahead'
    ], axis=1
)
df_energy.describe().round(2)

All nans for generation hydro pumped storage aggregated
All nans for forecast wind offshore eday ahead
All zeroes for generation fossil coal-derived gas
All zeroes for generation fossil oil shale
All zeroes for generation fossil peat
All zeroes for generation geothermal
All zeroes for generation marine
All zeroes for generation wind offshore


,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load actual,price day ahead,price actual
count,35045.00,35046.00,35046.00,35046.00,35045.00,35045.00,35045.00,35046.00,35047.00,35046.00,35046.00,35046.00,35045.00,35046.00,35028.00,35064.00,35064.00
mean,383.51,448.06,5622.74,4256.07,298.32,475.58,972.12,2605.11,6263.91,60.23,85.64,1432.67,269.45,5464.48,28696.94,49.87,57.88
std,85.35,354.57,2201.83,1961.60,52.52,792.41,400.78,1835.20,839.67,20.24,14.08,1680.12,50.20,3213.69,4574.99,14.62,14.20
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18041.00,2.06,9.33
25%,333.00,0.00,4126.00,2527.00,263.00,0.00,637.00,1077.25,5760.00,53.00,73.00,71.00,240.00,2933.00,24807.75,41.49,49.35
50%,367.00,509.00,4969.00,4474.00,300.00,68.00,906.00,2164.00,6566.00,57.00,88.00,616.00,279.00,4849.00,28901.00,50.52,58.02
75%,433.00,757.00,6429.00,5838.75,330.00,616.00,1250.00,3757.00,7025.00,80.00,97.00,2578.00,310.00,7398.00,32192.00,60.53,68.01
max,592.00,999.00,20034.00,8359.00,449.00,4523.00,2000.00,9728.00,7117.00,106.00,119.00,5792.00,357.00,17436.00,41015.00,101.99,116.80


In [17]:
df_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil gas                        35046 non-null  float64
 4   generation fossil hard coal                  35046 non-null  float64
 5   generation fossil oil                        35045 non-null  float64
 6   generation hydro pumped storage consumption  35045 non-null  float64
 7   generation hydro run-of-river and poundage   35045 non-null  float64
 8   generation hydro water reservoir             35046 non-null  float64
 9   generation nuclear                           35047 non-null  float64
 10

In [18]:
# convert time to datetime object and set index
df_energy["time"] = pd.to_datetime(df_energy["time"],utc=True,infer_datetime_format=True)
df_energy = df_energy.set_index("time")

In [20]:
# Find nans and duplicates in the set 
print("Total missing/NaN is: {}".format(
    df_energy.isnull().values.sum()
))
print("Total duplicates in data: {}".format(
    df_energy.duplicated(keep="first").sum()
))

Total missing/NaN is: 292
Total duplicates in data: 0


No duplicates.  
Nans -> fill with interpolation (important for time-seris)

In [ ]:
# 